# Import Libraries

In [97]:
import re
import json
import numpy
import pandas as pd

# Prep Data

#### Read in the raw data

In [98]:
raw_data = pd.read_json("Data.json")
raw_data.head()

,session_id,unix_timestamp,cities,user
0,[X061RFWB06K9V],[1442503708],"[New York NY, Newark NJ]","[[{'user_id': 2024, 'joining_date': '2015-03-2..."
1,[5AZ2X2A9BHH5U],[1441353991],"[New York NY, Jersey City NJ, Philadelphia PA]","[[{'user_id': 2853, 'joining_date': '2015-03-2..."
2,[SHTB4IYAX4PX6],[1440843490],[San Antonio TX],"[[{'user_id': 10958, 'joining_date': '2015-03-..."
3,[JBRB8MZGTX3M4],[1427268063],[Edmonton AB],"[[{'user_id': 7693, 'joining_date': '2015-03-1..."
4,[YJCMPURC2FL9C],[1430559067],"[Phoenix AZ, Houston TX]","[[{'user_id': 7506, 'joining_date': '2015-02-2..."


#### Convert the data we read in into a new Dataframe but extract the JSON structured data into simpler dataframes

In [170]:
data = pd.DataFrame(raw_data['session_id'].apply(lambda x : x[0]))
data['unix_timestamp'] = raw_data['unix_timestamp'].apply(lambda x: x[0])
data['cities'] = raw_data['cities'].apply(lambda x : re.compile(" *, *").split(x[0].strip()))
data['user_id'] = raw_data['user'].apply(lambda x : x[0][0]['user_id'])
data['joining_date'] = raw_data['user'].apply(lambda x : x[0][0]['joining_date'])
data['country'] = raw_data['user'].apply(lambda x : x[0][0]['country'].strip())
data.head()

,session_id,unix_timestamp,cities,user_id,joining_date,country
0,X061RFWB06K9V,1442503708,"[New York NY, Newark NJ]",2024,2015-03-22,UK
1,5AZ2X2A9BHH5U,1441353991,"[New York NY, Jersey City NJ, Philadelphia PA]",2853,2015-03-28,DE
2,SHTB4IYAX4PX6,1440843490,[San Antonio TX],10958,2015-03-06,UK
3,JBRB8MZGTX3M4,1427268063,[Edmonton AB],7693,2015-03-12,IT
4,YJCMPURC2FL9C,1430559067,"[Phoenix AZ, Houston TX]",7506,2015-02-28,UK


#### Cities is now a list, and all other fields have been flattened.
#### We can split up the cities list into a dataframe with columns for each session.

In [171]:
cities = data[['session_id', 'cities']]
cities.head()

,session_id,cities
0,X061RFWB06K9V,"[New York NY, Newark NJ]"
1,5AZ2X2A9BHH5U,"[New York NY, Jersey City NJ, Philadelphia PA]"
2,SHTB4IYAX4PX6,[San Antonio TX]
3,JBRB8MZGTX3M4,[Edmonton AB]
4,YJCMPURC2FL9C,"[Phoenix AZ, Houston TX]"


#### We can unpivot the list of cities into a list of sessions + cities

In [172]:
cities2 = cities['cities'].apply(pd.Series)
a = pd.concat([cities[:], cities2[:]], axis=1)
a.head()

,session_id,cities,0,1,2,3,4,5,6,7,8,9,10
0,X061RFWB06K9V,"[New York NY, Newark NJ]",New York NY,Newark NJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5AZ2X2A9BHH5U,"[New York NY, Jersey City NJ, Philadelphia PA]",New York NY,Jersey City NJ,Philadelphia PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHTB4IYAX4PX6,[San Antonio TX],San Antonio TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JBRB8MZGTX3M4,[Edmonton AB],Edmonton AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YJCMPURC2FL9C,"[Phoenix AZ, Houston TX]",Phoenix AZ,Houston TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
b = a.melt(id_vars=['session_id', 'cities'], value_name='city')
city_session = b.dropna()[['session_id', 'variable', 'city']].sort_values(['session_id', 'variable'])
city_session.head(20)

,session_id,variable,city
11246,0004TZ56GY6S7,0,San Diego CA
31268,0004TZ56GY6S7,1,New York NY
19702,004M08GLQ7ZXH,0,Toronto ON
5274,007O320DQ7AXT,0,New York NY
14910,007UIISVZJOW5,0,San Antonio TX
16667,008ZL2D1PLC4O,0,New York NY
36689,008ZL2D1PLC4O,1,Chicago IL
13779,00C8JBREV4NBU,0,Philadelphia PA
2931,00FBBMQ9RPM9N,0,Houston TX
2353,00H99K6WLXUXI,0,San Antonio TX


# Do Analysis

We can have a look at the data values. Given there are Canadian cities in the dataset, I would assume that Canada is missing. But hopefully they've provided a list of countries otherwise there are actually about 213 countries missing...

In [174]:
data['country'].unique()

array(['UK', 'DE', 'IT', 'ES', '', 'US', 'FR'], dtype=object)

We can have a look at the unique cities in our dataset.

In [175]:
city_session['city'].unique()

array(['San Diego CA', 'New York NY', 'Toronto ON', 'San Antonio TX',
       'Chicago IL', 'Philadelphia PA', 'Houston TX', 'Fort Worth TX',
       'Arlington TX', 'Oshawa ON', 'Boston MA', 'Montreal QC',
       'Hamilton ON', 'Buffalo NY', 'Saint Catharines Niagara ON',
       'Los Angeles CA', 'Jersey City NJ', 'Newark NJ', 'Indianapolis IN',
       'Long Beach CA', 'Vancouver BC', 'Columbus OH', 'Calgary AB',
       'Seattle WA', 'Portland OR', 'Victoria BC', 'Santa Ana CA',
       'Dallas TX', 'Kitchener ON', 'Baltimore MD', 'Austin TX',
       'Phoenix AZ', 'Plano TX', 'Jacksonville FL', 'Detroit MI',
       'OTTAWA ON', 'Riverside CA', 'Milwaukee WI', 'Madison WI',
       'Anaheim CA', 'Cincinnati OH', 'Fort Wayne IN', 'Edmonton AB',
       'Scottsdale AZ', 'Mesa AZ', 'San Jose CA', 'Corpus Christi TX',
       'Cleveland OH', 'Windsor ON', 'Norfolk VA', 'Lexington KY',
       'Bakersfield CA', 'Chandler AZ', 'Glendale AZ', 'WASHINGTON DC',
       'Quebec QC', 'Halifax NS', 'Toled

One of the really useful functions is group by... This transforms our list of cities by session into an unpivotted table. We can use this as the basis for our analysis... 1.0's flag where a city was searched ina session.

In [190]:
stack = c.groupby('session_id')['city'].value_counts().unstack().fillna(0)
stack.head(30)

city,Anaheim CA,Arlington TX,Atlanta GA,Austin TX,Bakersfield CA,Baltimore MD,Birmingham AL,Boston MA,Buffalo NY,Calgary AB,...,Toledo OH,Toronto ON,Tucson AZ,Tulsa OK,Vancouver BC,Victoria BC,Virginia Beach VA,WASHINGTON DC,Wichita KS,Windsor ON
session_id,,,,,,,,,,,,,,,,,,,,,
0004TZ56GY6S7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
004M08GLQ7ZXH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007O320DQ7AXT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007UIISVZJOW5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
008ZL2D1PLC4O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00C8JBREV4NBU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00FBBMQ9RPM9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00H99K6WLXUXI,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00KLXTMMVX344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Once we have our table above, we can go through each city (lets call this A) and figure out which was the most searched city when also searching for A. From the table, A is always going to be the highest because we are filtering on that city. We need to choose the second highest searched city. That's what we do below:

In [191]:
# go through each of the cities in our dataset (which is also the columns in our dataset)
for city in stack.columns:
    # for each city, filter the table by sessions which have used our city.
    substack = stack[stack[city] == 1.0]
    # sort by the sum of all cities in our subset.
    sorted_list = numpy.sum(substack, axis=0).sort_values(ascending=False)
    # pick the highest after removing the top result (which is always going to be our initial city, we want 2nd result.)
    highest = sorted_list.iloc[1:].idxmax() if len(sorted_list) > 1 else 'Usually searched alone'
    # print the data... or do whatever you want to do with it...
    print("City [{}]: {}".format(city, highest))

City [Anaheim CA]: Los Angeles CA
City [Arlington TX]: Dallas TX
City [Atlanta GA]: Jacksonville FL
City [Austin TX]: Houston TX
City [Bakersfield CA]: Los Angeles CA
City [Baltimore MD]: New York NY
City [Birmingham AL]: Atlanta GA
City [Boston MA]: New York NY
City [Buffalo NY]: Toronto ON
City [Calgary AB]: Vancouver BC
City [Chandler AZ]: Phoenix AZ
City [Charlotte NC]: Jacksonville FL
City [Chesapeake VA]: New York NY
City [Chicago IL]: New York NY
City [Cincinnati OH]: Indianapolis IN
City [Cleveland OH]: Detroit MI
City [Columbus OH]: Cincinnati OH
City [Corpus Christi TX]: San Antonio TX
City [Dallas TX]: Plano TX
City [Detroit MI]: Windsor ON
City [Edmonton AB]: Calgary AB
City [Fort Wayne IN]: Chicago IL
City [Fort Worth TX]: Dallas TX
City [Fresno CA]: San Jose CA
City [Glendale AZ]: Phoenix AZ
City [Greensboro NC]: Montreal QC
City [Halifax NS]: Halifax NS
City [Hamilton ON]: Toronto ON
City [Hialeah FL]: Hialeah FL
City [Houston TX]: New York NY
City [Indianapolis IN]: Cin